In [6]:
import pandas as pd
import tensorflow as tf
%pip install datasets
%pip install transformers
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModel
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Feb 25 15:18:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# see number of processors
import multiprocessing
multiprocessing.cpu_count()

12

In [9]:
# load yelp review dataset
dataset = load_dataset('yelp_polarity', split='train')
dataset

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'label'],
    num_rows: 560000
})

In [10]:
dataset_split = dataset.train_test_split(test_size=0.2)

In [11]:
dataset_split['train'][0]

{'text': "Well, never turned out to be about 10 months lol. My husband and I have tried this place a couple times since the awful first experience because it's right next to our new eye doctor and has happy hour all day. \\n\\nWe went mid afternoon both times and they weren't at all busy. That didn't make the service any better. I guess that's what happens when you give your wait staff their own section of the restaurant with a huge  flat screen.\\n\\nThe first time we ordered a couple Vegas rolls, some sashimi and a shrimp tempura roll. My husband asked for no wasabi on the sashimi, it came with wasabi. I asked for no cucumber on my tempura roll and it, of course, had cucumber. My hubby really liked the Vegas rolls, I thought they were ok. If you're having something with carrots make sure to order them without as their carrots taste nasty, like they soak them in something before serving.\\n\\nWe only went the second time because my hubby's favorite sushi place is closed on Mondays. Th

In [12]:
dataset_split = dataset_split.flatten()
dataset_split['train'][0]

{'text': "Well, never turned out to be about 10 months lol. My husband and I have tried this place a couple times since the awful first experience because it's right next to our new eye doctor and has happy hour all day. \\n\\nWe went mid afternoon both times and they weren't at all busy. That didn't make the service any better. I guess that's what happens when you give your wait staff their own section of the restaurant with a huge  flat screen.\\n\\nThe first time we ordered a couple Vegas rolls, some sashimi and a shrimp tempura roll. My husband asked for no wasabi on the sashimi, it came with wasabi. I asked for no cucumber on my tempura roll and it, of course, had cucumber. My hubby really liked the Vegas rolls, I thought they were ok. If you're having something with carrots make sure to order them without as their carrots taste nasty, like they soak them in something before serving.\\n\\nWe only went the second time because my hubby's favorite sushi place is closed on Mondays. Th

In [13]:
# Load the GPT-2 tokenizer which uses a byte-level Byte Pair Encoding (BPE) algorithm to break down input text into a sequence of subword units.
# It is designed to handle large amounts of text data and to be highly efficient
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Load the GPT-2 model
bert = TFAutoModel.from_pretrained("distilgpt2")

# Define a preprocessing function that tokenizes the text using the tokenizer
# and joins the tokens back together into a string
def preprocess_function(examples):
    return tokenizer(["".join(x) for x in examples["text"]], truncation=True)

# Apply the preprocessing function to the Yelp dataset using the `map` method
# `batched=True` allows the dataset to be processed in batches, reducing memory usage
# `num_proc=8` parallelizes the preprocessing across 8 processes for faster execution
# `remove_columns=dataset_split["train"].column_names` removes the columns from the dataset that are not needed for further processing
tokenized_yelp = dataset_split.map(
    preprocess_function,
    batched=True,
    num_proc=12,
    remove_columns=dataset_split["train"].column_names,
)


All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


Map (num_proc=12):   0%|          | 0/448000 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/112000 [00:00<?, ? examples/s]

In [14]:
# Set the maximum block size for the text data
block_size = 128

# Define a function to group input examples into blocks of size block_size
# The function concatenates the input examples into a single list and then
# divides the list into blocks of size block_size
def group_texts(examples):
    # Concatenate the examples into a single list
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    
    # Compute the total length of the concatenated examples
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # Round the total length down to the nearest multiple of block_size
    total_length = (total_length // block_size) * block_size
    
    # Divide the concatenated examples into blocks of size block_size
    # for each key in the input examples
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    
    # Set the labels for each block to be the same as the input_ids
    result["labels"] = result["input_ids"].copy()
    
    # Return the resulting blocks
    return result


In [15]:
lm_dataset = tokenized_yelp.map(group_texts, batched=True, num_proc=12)

Map (num_proc=12):   0%|          | 0/448000 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/112000 [00:00<?, ? examples/s]

In [16]:
# Import the necessary classes from the Transformers library
from transformers import TFAutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling

# Load the pre-trained GPT-2 model for causal language modeling
model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

# Create a data collator for language modeling using the loaded tokenizer
# The collator is used to prepare the input data for training the language model
# The `mlm` flag is set to `False` to indicate that the model should perform causal language modeling rather than masked language modeling
# The `return_tensors` parameter is set to `tf` to ensure that the collator returns TensorFlow tensors
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [17]:
# Set the padding token for the tokenizer to be the end-of-sequence (eos) token
tokenizer.pad_token = tokenizer.eos_token

# Prepare the training set as a TensorFlow dataset
# The `prepare_tf_dataset` method prepares a dataset of input sequences for training the language model
# The `lm_dataset["train"]` argument provides the training set as a `Dataset` object
# The `shuffle` parameter is set to `True` to shuffle the examples in the dataset
# The `batch_size` parameter sets the number of examples in each training batch
# The `collate_fn` parameter specifies the data collator to use to prepare the input data for training
tf_train_set = model.prepare_tf_dataset(
    lm_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

# Prepare the test set as a TensorFlow dataset
# The `prepare_tf_dataset` method prepares a dataset of input sequences for evaluation of the language model
# The `lm_dataset["test"]` argument provides the test set as a `Dataset` object
# The `shuffle` parameter is set to `False` to keep the examples in the original order
# The `batch_size` parameter sets the number of examples in each evaluation batch
# The `collate_fn` parameter specifies the data collator to use to prepare the input data for evaluation
tf_test_set = model.prepare_tf_dataset(
    lm_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [18]:
# see first 5 tokenized reviews
tf_train_set.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 128), dtype=tf.int64, name=None))>

In [19]:
# Load the pre-trained GPT-2 model for causal language modeling
not_tuned_model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

# Compile the not_tuned_model
# Compiling the model is required to initialize the optimizer and other parameters
not_tuned_model.compile()

# Evaluate the not_tuned_model on the test set
# The `evaluate` method evaluates the performance of the model on a given dataset
# The `tf_test_set` argument provides the test set as a TensorFlow dataset
not_tuned_model.evaluate(tf_test_set.take(100))


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


100/100 [==============================] - 7s 17ms/step - loss: 4.3131


4.313097953796387

In [21]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

model.compile(optimizer=optimizer)

history = model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


37971/37971 [==============================] - 1767s 46ms/step - loss: 3.6248 - val_loss: 3.4467


In [22]:
model.evaluate(tf_test_set)

9509/9509 [==============================] - 159s 17ms/step - loss: 3.4467


3.446709632873535

In [23]:
from transformers import pipeline

generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

generator("I love this restaurant")

/usr/local/lib/python3.8/dist-packages/transformers/generation/tf_utils.py:603: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation/tf_utils.py:702: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 50 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'I love this restaurant as well. The prices are reasonable at their most basic (I think $5 for 1 egg = a little more than half the cost). \\n\\nWe were seated by a friendly man. He made us feel special and'}]

In [24]:
generator("I'm not so sure about")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I'm not so sure about the prices. I had a good bowl of the salad and it tasted good. It was good. I was on a late lunch break with the other guy.\\n\\nI ordered the green chile with lime"}]